In [26]:
import pandas as pd
import numpy as np
from numpy.random import seed, randint
import math

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl

Synthesize data

In [31]:
P_true = 0.2
N = 30

def do_experiment():
     return np.random.binomial(N, P_true, 100)

print("Example data with true parameter: {}".format(do_experiment()))

Example data with true parameter: [ 7  8  8  7  5  2  7  4  8  9  4  5  7  9  3  7  5  5  5  6  9  3  3  7
  2  9  6  4  5  7  5  9  9  4  7  6  5  7  5  5  7  6  7  6  5 10  8  6
  6  4  6  7  5  9  3  8  7  6  8  9  2  7  7  3  9  4  5  2  5  7  5  3
  2  2  5  5  3  4  6  5  4  1  8  3  8  8 10  2  6  8 14  4  6  7  7  9
  4  4  5  3]


# Bayesian inference, iteration scheme

**Iteration step**

In [13]:
def bayes(data, alpha, beta):
    n = len(data)
    new_alpha = alpha + np.sum(data)
    new_beta = beta + (N * (len(data))) - np.sum(data) 
    new_p = alpha / (alpha + beta)
    return new_alpha, new_beta, new_p

**Iteration scheme**

In [29]:
n_iter = 100
alpha = 1
beta = 1
estimated_p = 0
traces = []
for i in range(0, n_iter):
    # synthesize data
    data = do_experiment()
    # inference
    alpha, beta, estimated_p = bayes(data, alpha, beta)
    traces.append((alpha, beta, estimated_p))
print("Last estimated p={}, distance to true p={}".format(estimated_p, abs(estimated_p - P_true)))

Last estimated p=0.2006249116167568, distance to true p=0.0006249116167567903


**Showing the convergence to true parameter P**

In [32]:
import pymc3 as pm
for tr in traces:
    posterior_sample = np.random.beta(tr[0], tr[1], 10000000)
    # TODO: explain how HPD/BCI is calculated
    # TODO: reimplement. It is not really complicated
    hpd = pm.stats.hpd(posterior_sample, credible_interval= 0.90)
    print("HPD={}, width={}, estimated_p={}, distance={}".format(
        hpd,
        abs(hpd[0] - hpd[1]),
        tr[2],
        abs(tr[2] - P_true)
    ))

HPD=[0.18491977 0.20879019], width=0.023870412651989276, estimated_p=0.5, distance=0.3
HPD=[0.19521405 0.2123171 ], width=0.017103043648654975, estimated_p=0.19686875416389074, distance=0.00313124583610927
HPD=[0.19321956 0.20709097], width=0.01387140677594445, estimated_p=0.20376541152949018, distance=0.0037654115294901702
HPD=[0.19114896 0.20309614], width=0.011947182601082906, estimated_p=0.20017773828038213, distance=0.00017773828038211703
HPD=[0.19195613 0.2026449 ], width=0.010688769572789747, estimated_p=0.19713381103149474, distance=0.0028661889685052677
HPD=[0.19211139 0.20186312], width=0.009751730562028105, estimated_p=0.1973070257299027, distance=0.0026929742700973203
HPD=[0.19486507 0.20393088], width=0.009065809248625739, estimated_p=0.19697811354293968, distance=0.0030218864570603354
HPD=[0.19652788 0.2050341 ], width=0.008506216042883818, estimated_p=0.1994095800399962, distance=0.0005904199600038074
HPD=[0.19546602 0.2034656 ], width=0.007999574914895474, estimated_p=0

HPD=[0.19932543 0.20220075], width=0.0028753176035226924, estimated_p=0.2006405735210288, distance=0.0006405735210287811
HPD=[0.19932857 0.20218341], width=0.0028548405301485336, estimated_p=0.2007599927619737, distance=0.0007599927619736935
HPD=[0.19931566 0.20215105], width=0.0028353898387173393, estimated_p=0.2007539835306711, distance=0.0007539835306710763
HPD=[0.19940748 0.20222303], width=0.0028155494682267634, estimated_p=0.20073888204738846, distance=0.0007388820473884505
HPD=[0.19945143 0.20224835], width=0.0027969219061243222, estimated_p=0.20081551766650532, distance=0.000815517666505311
HPD=[0.19956255 0.20234228], width=0.002779726033020069, estimated_p=0.20084954189601895, distance=0.0008495418960189349
HPD=[0.19958058 0.20234165], width=0.0027610691874958293, estimated_p=0.20095376929982844, distance=0.0009537692998284331
HPD=[0.19952594 0.20226851], width=0.00274256846638965, estimated_p=0.2009631494460575, distance=0.000963149446057493
HPD=[0.19954859 0.20227254], widt

In [20]:
import scipy.stats as stats
stats.bayes_mvs(posterior_sample, 0.95)

(Mean(statistic=0.1996889863878346, minmax=(0.19968853414838467, 0.19968943862728455)),
 Variance(statistic=5.324032604223419e-07, minmax=(5.319365969639052e-07, 5.328699238807786e-07)),
 Std_dev(statistic=0.0007296596880891406, minmax=(0.0007293399065073605, 0.0007299794696709207)))